# Working with Networks - Quickstart
## OpenStreetMap - Shortest Paths - Reachable Areas

In this notebook we want to play around with the osmnx and networkx libraries. In specific we will see how we load OpenStreetMap data, get locations of amenities e.g. restaurants and calculate routes and walking distances to this restaurants.

First we start with the import of all different libraries we are going to use:

In [34]:
import geopandas as gpd
import shapely
import folium
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import numpy as np
import os
import pandas as pd



Then we have to import our study area. For this example we will use Bremen and in particular the district Westend again. Note that we will work with Coordinate Reference Systems (CRS) in this example. There a thousands, of different reference systems but gladly we can differ them by unique, numerical, so called epsg-codes. We will need two types of reference systems: 

1: The WGS84 - World Geodetic System 1984 which is also used for GPS and OSM for example. It is based on latitude and longitude in degrees. The epsg code is 4326.

2: The ETRS89-extended / LAEA Europe System, which is a projected reference system in meters focused on the european continent. The epsg code is 3035.

Firstly lets import and check the projection of our study area:

In [39]:
#os.chdir("...")
ffm = gpd.read_file("../data/3_Neighborhoods/Neighborhoods_Frankfurt_am_Main.gpkg")
ffm.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

We see that the projection of our study area is in ETRS89 with the epsg code 3035. But since OSM works with the WGS 84 - epsg:4326 we have to reproject our file. Then we select only the district of Westend and create a starting point for the routing algorithms. Also note that there is a difference between a GeoSeries and a GeoDataFrame.

In [40]:
pd.set_option('display.max_columns', None)

ffm.head(10)

,Neighborhood_FID,Neighborhood_Name,City_Code,City_Name,geometry
0,1,Altstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4226384.447 3000227.322, 42263..."
1,2,Innenstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227082.463 3001711.312, 42272..."
2,3,Bahnhofsviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225284.051 3000486.630, 42252..."
3,4,Westend-Süd,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225219.883 3002076.156, 42252..."
4,5,Westend-Nord,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225993.384 3003603.082, 42260..."
5,6,Nordend-West,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227858.464 3003565.575, 42278..."
6,7,Nordend-Ost,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227867.212 3001590.060, 42277..."
7,8,Ostend,06412000,Frankfurt am Main,"MULTIPOLYGON (((4231428.842 3001758.583, 42314..."
8,9,Bornheim,06412000,Frankfurt am Main,"MULTIPOLYGON (((4228434.896 3001994.720, 42284..."
9,10,Gutleutviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4222367.116 2998542.440, 42223..."


In [20]:
ffm = ffm.to_crs(epsg = 4326)
#westend = berlin.loc[bremen.Neighborhood_Name == "Westend"]
start_point = gpd.GeoSeries(shapely.Point([8.788266, 53.094551]))



In [73]:
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_colwidth')
#pd.set_option('display.max_colwidth', None)
ffm.head(100)

,Neighborhood_FID,Neighborhood_Name,City_Code,City_Name,geometry,centr
0,1,Altstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
1,2,Innenstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
2,3,Bahnhofsviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
3,4,Westend-Süd,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
4,5,Westend-Nord,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
5,6,Nordend-West,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227858.46392 3003565.57485, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
6,7,Nordend-Ost,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227867.21204 3001590.05997, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
7,8,Ostend,06412000,Frankfurt am Main,"MULTIPOLYGON (((4231428.84153 3001758.58257, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
8,9,Bornheim,06412000,Frankfurt am Main,"MULTIPOLYGON (((4228434.89586 3001994.71993, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...
9,10,Gutleutviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4222367.11615 2998542.43979, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...


In [89]:
import geopandas as gpd

ffm["NEIGHBORS"] = None


for index, country in ffm.iterrows():   

    # get 'not disjoint' countries
    neighbors = ffm[~ffm.geometry.disjoint(country.geometry)].Neighborhood_Name.tolist()

    # remove own name of the country from the list
    neighbors = [ name for name in neighbors if country.Neighborhood_Name != name ]

    # add names of neighbors as NEIGHBORS value
    ffm.at[index, "NEIGHBORS"] = ", ".join(neighbors)
   

In [90]:
ffm.head(100)

,Neighborhood_FID,Neighborhood_Name,City_Code,City_Name,geometry,dist,NEIGHBORS
0,1,Altstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4...",0.000000,"Innenstadt, Sachsenhausen-N."
1,2,Innenstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4...",0.000000,"Altstadt, Bahnhofsviertel, Westend-Süd, Norden..."
2,3,Bahnhofsviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4...",0.000000,"Innenstadt, Westend-Süd, Gutleutviertel, Gallu..."
3,4,Westend-Süd,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4...",0.000000,"Innenstadt, Bahnhofsviertel, Westend-Nord, Nor..."
4,5,Westend-Nord,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4...",304.484170,"Westend-Süd, Nordend-West, Bockenheim, Dornbusch"
5,6,Nordend-West,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227858.46392 3003565.57485, 4...",0.000000,"Innenstadt, Westend-Süd, Westend-Nord, Nordend..."
6,7,Nordend-Ost,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227867.21204 3001590.05997, 4...",0.000000,"Innenstadt, Nordend-West, Ostend, Bornheim"
7,8,Ostend,06412000,Frankfurt am Main,"MULTIPOLYGON (((4231428.84153 3001758.58257, 4...",0.000000,"Innenstadt, Nordend-Ost, Bornheim, Sachsenhaus..."
8,9,Bornheim,06412000,Frankfurt am Main,"MULTIPOLYGON (((4228434.89586 3001994.71993, 4...",864.461323,"Nordend-West, Nordend-Ost, Ostend, Preungeshei..."
9,10,Gutleutviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4222367.11615 2998542.43979, 4...",504.739289,"Bahnhofsviertel, Gallus, Sachsenhausen-N., Nie..."


In [70]:
from shapely.geometry import Polygon, Point
import geopandas

In [71]:
ffm['centr'] = ffm['geometry'].iloc[1]

In [74]:
t1 = ffm['geometry']
t = geopandas.GeoSeries(t1)
t.crs = 4326

centr = ffm['centr']
centr = geopandas.GeoSeries(centr)
centr.crs = 4326

dist = t.distance(centr)
print(dist)

ffm['dist'] = t.distance(centr)

0         0.000000
1         0.000000
2         0.000000
3         0.000000
4       304.484170
5         0.000000
6         0.000000
7         0.000000
8       864.461323
9       504.739289
10      546.207530
11     1264.784949
12        0.000000
13      640.851124
14     5951.809338
15     1591.473167
16     1696.812538
17     3593.729542
18     3948.851043
19     3494.421124
20     3196.167871
21     3812.648932
22     4246.426282
23     5303.116846
24     2423.967654
25     1844.320695
26     3370.616422
27     2077.760902
28     2462.826781
29     6483.516243
30     4743.222252
31     2297.532460
32     2484.004232
33     3523.290508
34     7289.082031
35     4935.111309
36     9974.276159
37    10474.199700
38     8364.943379
39     5019.081135
40     8080.356866
41     6144.655637
42     5656.961170
43     7468.093283
44     4354.260734
45     4710.998237
dtype: float64


/var/folders/n5/wh899bt53vx2wn7f_79lm0lr0000gn/T/ipykernel_6367/3195193752.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dist = t.distance(centr)
/var/folders/n5/wh899bt53vx2wn7f_79lm0lr0000gn/T/ipykernel_6367/3195193752.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ffm['dist'] = t.distance(centr)


In [80]:
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_colwidth')
#pd.set_option('display.max_colwidth', None)
ffm.head(100)

,Neighborhood_FID,Neighborhood_Name,City_Code,City_Name,geometry,centr,dist
0,1,Altstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4226384.44674 3000227.32158, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,0.000000
1,2,Innenstadt,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227082.46278 3001711.31224, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,0.000000
2,3,Bahnhofsviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225284.05145 3000486.62977, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,0.000000
3,4,Westend-Süd,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225219.88286 3002076.15554, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,0.000000
4,5,Westend-Nord,06412000,Frankfurt am Main,"MULTIPOLYGON (((4225993.38385 3003603.08203, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,304.484170
5,6,Nordend-West,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227858.46392 3003565.57485, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,0.000000
6,7,Nordend-Ost,06412000,Frankfurt am Main,"MULTIPOLYGON (((4227867.21204 3001590.05997, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,0.000000
7,8,Ostend,06412000,Frankfurt am Main,"MULTIPOLYGON (((4231428.84153 3001758.58257, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,0.000000
8,9,Bornheim,06412000,Frankfurt am Main,"MULTIPOLYGON (((4228434.89586 3001994.71993, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,864.461323
9,10,Gutleutviertel,06412000,Frankfurt am Main,"MULTIPOLYGON (((4222367.11615 2998542.43979, 4...",MULTIPOLYGON (((4227082.462780299 3001711.3122...,504.739289


In [84]:
del ffm['centr']

ffm.to_file("../data/ffm_dist.gpkg", driver="GPKG")



KeyError: 'centr'

In [29]:
import shapely.geometry
import shapely.wkt